# Initialize and Run Iterative Biome-BGC Muso 5 Model
### Use Kernel Python 3.8

This notebook walks through the process of reading in all pre-processed information generated in previous steps and running an geographically-iterative, parallelized version of Biome-BGC Muso.

## Set the Project Directory and Read in Def Files

### Set the Project Directory and Set up Notebook

<div class="alert alert-block alert-danger">
<b> Project Name Setting: </b> Please set the Project Name for your project </div>

In [ ]:
computing_id = ''

# CHANGE PROJECT NAME HERE
PROJECT_NAME = ""

# CURRENT DIRECTORY HERE
CURRENT_DIRECTORY = f"/scratch/{computing_id}/Biome-bgc-new"

# MODEL RESOLUTION
RESOLUTION = 30

In [ ]:
import os, shutil
script_dir = os.path.join(os.getcwd(),'MODEL.py')
os.chdir(CURRENT_DIRECTORY)

### Install and Load Slurm Magic

In [ ]:
!pip install git+https://github.com/NERSC/slurm-magic.git

In [ ]:
%load_ext slurm_magic

### Create Slurm Job to Run Model

The following cell contains the script needed to submit a slurm job. There are a few things that you will need to change. First, please change the number of cores to your desired amount. The larger the area of simulation, the more cores you will need. A good rule of thumb is to use one core per Daymet cell being simulated up to the maximum amount (40). As it is currently set up, this model is limited to only 1 node for the time being.

Second, change the runtime to an adequate amount. One daymet patch on one core running daily for 40 years takes ~30 minutes. Also change the allocation and mail settings.

Lastly, change the final command found in this script. The format should match that show below and have the model variables shown in brackets in place.

singularity exec /share/resources/containers/singularity/rhessys/rhessys-leb3t.sif python {CURRENT_DIRECTORY}/MODEL.py {MODEL_NAME} {CURRENT_DIRECTORY} {FREQUENCY (daily, monthly_agg, or annual_agg)} {CELL_SIZE}



In [ ]:
job_name = PROJECT_NAME

with open(os.path.join(CURRENT_DIRECTORY,job_name+'.job'),'w') as f:
    f.writelines('#!/bin/bash\n')
    f.writelines('#SBATCH --partition=instructional\n')
    f.writelines('#SBATCH --nodes=1\n')
    f.writelines('#SBATCH --ntasks-per-node=12\n')
    f.writelines('#SBATCH -t 1:00:00\n')
    f.writelines(f'#SBATCH --output={job_name}.out\n')
    f.writelines(f'#SBATCH --error={job_name}.err\n')
    f.writelines(f'#SBATCH --job-name={job_name}\n')
    f.writelines('#SBATCH -A evhy5700-leb3t\n')
    f.writelines(f'#SBATCH --mail-user={computing_id}@virginia.edu\n')
    f.writelines(f'#SBATCH --mail-type=ALL\n')

    f.writelines('module purge\n')
    f.writelines('module load singularity\n')

    f.writelines(f'singularity exec /share/resources/containers/singularity/rhessys/rhessys-leb3t.sif python {script_dir} {PROJECT_NAME} {CURRENT_DIRECTORY} daily {RESOLUTION}\n')

In [ ]:
# !rm {os.path.join(CURRENT_DIRECTORY, PROJECT_NAME, 'model', 'ini_file','gridcells_complete.txt')}   ## Uncomment if you run the model again

In [ ]:
%sbatch {job_name}.job

In [ ]:
%squeue -u {computing_id}